In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "GPU bulunamadı")

True
NVIDIA RTX A5000


In [ ]:
import os
os.environ["HUGGING_FACE_HUB_TOKEN"] = "senin token"  # Hugging Face üzerinden ince ayar için kullanacağımız modeli indirmek için Hugging Face tokenı gerekmektedir

In [ ]:
# Gerekli kütüphaneyi içe aktarma
from unsloth import FastLanguageModel  # Unsloth kütüphanesinden hızlı model yükleme sınıfını içe aktarır, büyük dil modellerini optimize eder

# PyTorch kütüphanesini içe aktarma
import torch                           # PyTorch'u tensör işlemleri ve model hesaplamaları için içe aktarır

# Maksimum dizi uzunluğu ayarı
max_seq_length = 2048                  # Modelin bir seferde işleyebileceği maksimum token sayısını 2048 olarak tanımlar, RoPE Scaling ile otomatik ölçeklenir

# Veri tipi ayarı
#dtype = None                           # Veri tipini otomatik algılamaya bırakır; Tesla T4/V100 için Float16, Ampere+ için Bfloat16 seçilir
dtype = torch.bfloat16 #A5000 ekran kartı için bunu seçtik.

# 4-bit kuantizasyon ayarı
load_in_4bit = True                    # Modeli 4-bit kuantizasyon ile yükler, bu bellek kullanımını azaltır ve performansı korur (False yapılırsa tam hassasiyet kullanılır)

# 4-bit önceden kuantize edilmiş modellerin listesi
fourbit_models = [                     # Unsloth’un desteklediği, 4-bit kuantize edilmiş modellerin listesi; hızlı indirme ve bellek tasarrufu sağlar
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 8B modeli, 15 trilyon token ile eğitilmiş, 2 kat hızlı
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",  # Llama-3.1 8B’nin talimatlara özel sürümü
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",     # Llama-3.1 70B modeli
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # Llama-3.1 405B modeli, 4-bit olarak yükleniyor
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # Mistral Nemo temel modeli, 12B, 2 kat hızlı
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",  # Mistral Nemo’nun talimat sürümü
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral 7B v0.3, 2 kat hızlı
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",   # Mistral 7B v0.3 talimat sürümü
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 mini, talimatlara özel, 2 kat hızlı
    "unsloth/Phi-3-medium-4k-instruct",        # Phi-3 orta boy, 4k token kapasiteli
    "unsloth/gemma-2-9b-bnb-4bit",             # Gemma 2 9B modeli
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2 27B modeli, 2 kat hızlı
]  # Daha fazla model için: https://huggingface.co/unsloth
model_path = "C:\\Users\\Mehmet\\Desktop\\Denizhan\\model_egitim\\merged_model"

# Modeli ve tokenizer'ı önceden eğitilmiş olarak yükleme
model, tokenizer = FastLanguageModel.from_pretrained( 
   # force_download=True,
      # Modeli ve tokenizer'ı yükler, Unsloth optimizasyonlarıyla
    model_name=model_path,   # Yüklenecek modelin adı: LLaMA-3.1 8B (8 milyar parametre)
    max_seq_length=max_seq_length,            # Maksimum dizi uzunluğunu 2048 olarak ayarlar
    dtype=dtype,                              # Veri tipini otomatik algılamaya bırakır (Tesla T4 için Float16 olur)
    load_in_4bit=load_in_4bit,                # 4-bit kuantizasyonu etkinleştirir, belleği optimize eder
    #token = "senin token",                       # Eğer kısıtlı erişimli bir model kullanıyorsanız Hugging Face token’ı eklenir (burada pasif)
)  # Model ve tokenizer nesnelerini döndürür

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


C:\Users\Mehmet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA RTX A5000. Max memory: 23.988 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.12s/it]


In [4]:
CHECKPOINT_DIR =  "C:\\Users\\Mehmet\\Desktop\\Denizhan\\model_egitim2" #Model verilerini kayıt edeceğiniz klasörü seçin.

In [ ]:
# Alpaca formatı için prompt şablonu tanımlama
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""  # Üç bölümden oluşan bir şablon: talimat (instruction), giriş (input) ve cevap (response) yer tutucuları içerir

# EOS_TOKEN tanımlama
EOS_TOKEN = tokenizer.eos_token  # Tokenizer'dan EOS (End of Sequence) token'ını alır, metnin sonunu işaretler

# Veri setini biçimlendirme fonksiyonu
def formatting_prompts_func(examples):  # Veri setindeki örnekleri Alpaca formatına dönüştürmek için bir fonksiyon
    instructions = examples["instruction"]  # Veri setindeki "instruction" sütununu alır (talimatlar)
    inputs       = examples["input"]        # Veri setindeki "input" sütununu alır (girişler)
    outputs      = examples["output"]       # Veri setindeki "output" sütununu alır (cevaplar)
    texts = []                              # Biçimlendirilmiş metinleri saklamak için boş bir liste oluşturur
    for instruction, input, output in zip(instructions, inputs, outputs):  # Her bir talimat, giriş ve cevap üçlüsünü eşleştirir
        # Alpaca şablonunu doldurur ve EOS_TOKEN ekler, yoksa üretim sonsuza dek devam eder!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN  # Şablonu doldurur ve metnin sonuna EOS token’ı ekler
        texts.append(text)                  # Biçimlendirilmiş metni listeye ekler
    return { "text" : texts, }              # "text" anahtarıyla biçimlendirilmiş metinleri bir sözlük olarak döndürür
pass                                       # Boş bir "pass" ifadesi (gereksiz, fonksiyon zaten tamamlandı)

# Veri setini yükleme ve işleme
from datasets import load_dataset       # Hugging Face’in datasets kütüphanesini içe aktarır
dataset = load_dataset("json", data_files="C:\\Users\\Mehmet\\Desktop\\Denizhan\\alpaca_format.json")["train"]  # JSON dosyasından veri setini yükler ve "train" bölümünü alır
dataset = dataset.map(formatting_prompts_func, batched=True,)  # Veri setine biçimlendirme fonksiyonunu toplu (batched) şekilde uygular

In [5]:
# Daha sonra tekrar kullanmak için:
from datasets import load_from_disk
dataset = load_from_disk("C:\\Users\\Mehmet\\Desktop\\Denizhan\\model_egitim\\final_model\\tokenized_dataset")

In [6]:
# Veri setinden bir örnek yazdır
print(dataset[0])

{'input_ids': [128000, 39314, 374, 459, 7754, 430, 16964, 264, 3465, 11, 35526, 449, 459, 1988, 430, 5825, 4726, 2317, 13, 9842, 264, 2077, 430, 36001, 45695, 279, 1715, 382, 14711, 30151, 512, 10351, 23684, 2322, 7907, 104250, 4355, 382, 14711, 5688, 512, 605, 13, 69657, 88, 2448, 18126, 120764, 24277, 809, 117545, 198, 68786, 5320, 123615, 9736, 24082, 1543, 58183, 108780, 113153, 5320, 14634, 352, 3458, 4265, 14966, 113086, 58804, 2448, 48497, 2448, 198, 68786, 5320, 123615, 9736, 24082, 1543, 58183, 108780, 113153, 5320, 14634, 352, 3458, 4265, 14966, 113086, 58804, 2448, 48497, 2448, 17773, 100805, 55809, 1008, 9655, 277, 80726, 480, 483, 4265, 68, 40528, 111508, 58112, 101766, 767, 1270, 112353, 104383, 24277, 587, 462, 69657, 88, 2448, 18126, 3209, 13532, 110560, 1543, 11, 330, 98581, 318, 127113, 112839, 119666, 64010, 101492, 26, 120781, 513, 10176, 88, 15686, 103611, 94582, 45356, 295, 74853, 13, 480, 30808, 5888, 305, 8115, 982, 1048, 10167, 318, 101149, 258, 102219, 55598, 

In [7]:
# Gerekli kütüphaneleri içe aktarma
from trl import SFTTrainer              # TRL kütüphanesinden SFTTrainer’ı içe aktarır, denetimli ince ayar için kullanılır
from transformers import TrainingArguments  # Hugging Face Transformers’tan eğitim argümanlarını içe aktarır
from unsloth import is_bfloat16_supported   # Unsloth’tan bfloat16 desteğini kontrol eden bir fonksiyon alır

# Eğitim sürecini başlatmak için trainer nesnesi oluşturma
trainer = SFTTrainer(                   # Modeli eğitmek için SFTTrainer nesnesi oluşturur
    model=model,                        # Eğitilecek model (önceki adımlarda LoRA ile yapılandırılmış)
    tokenizer=tokenizer,                # Modelin tokenizer’ı (metni token’lara çevirir)
    train_dataset=dataset,              # Eğitim için kullanılacak veri seti (Alpaca formatında işlenmiş)
    dataset_text_field=None,          # Veri setindeki hangi sütunun kullanılacağı: "text" (biçimlendirilmiş prompt’lar)
    max_seq_length=2048,                # Maksimum dizi uzunluğu (önceki adımda 2048 olarak tanımlı, uzun bağlamlar için yeterli)
    dataset_num_proc=2,                 # Veri işleme için kullanılacak işlemci sayısı: 4 (RTX A5000’in gücüyle paralel işleme artırıldı, 5 milyon satır için daha hızlı)
    #packing=False,                       # Veri paketlemeyi etkinleştirir; kısa diziler için 5x hız artışı sağlar, büyük veri setinde faydalı
    args=TrainingArguments(             # Eğitim argümanlarını tanımlar
        per_device_train_batch_size=12,  # Her cihaz (GPU) için eğitim batch boyutu: 8 (24 GB VRAM ile mümkün, performans artırıldı)
        gradient_accumulation_steps=4,  # Gradyan biriktirme adımları: 4 (efektif batch size’ı 8*4=32 yapar, bellek yönetimi için)
        warmup_steps=100,               # Öğrenme oranı ısınma adımları: 100 (büyük veri seti için daha yavaş ve stabil ısınma, 5’ten artırıldı)
        num_train_epochs=2,           # Tam bir epoch için eğitim (yorum satırı, şu anda kullanılmıyor, veri seti büyük olduğu için adım bazlı tercih ediliyor)
        #max_steps=1000,                 # Toplam eğitim adımı sayısı: 1000 (60’tan artırıldı, 5 milyon satır için yeterli öğrenme sağlamak için)
        learning_rate=2e-4,             # Öğrenme oranı: 0.0002 (modelin ne kadar hızlı öğreneceğini belirler, büyük veri setinde uygun)
        fp16=not is_bfloat16_supported(),  # Float16 kullanımı: bfloat16 desteklenmiyorsa True (RTX A5000’de bfloat16 destekleniyor)
        bf16=is_bfloat16_supported(),   # Bfloat16 kullanımı: destekleniyorsa True (Ampere mimarisi için RTX A5000’de etkin, daha verimli)
        logging_steps=10,               # Log kayıt sıklığı: her 10 adımda bir (1’den artırıldı, büyük veri setinde log sıklığını azaltır)
        optim="adamw_8bit",             # Optimize edici: 8-bit AdamW (bellek verimli bir versiyon, değişmedi)
        weight_decay=0.01,              # Ağırlık çürümesi: 0.01 (overfitting’i önlemek için regularization, değişmedi)
        lr_scheduler_type="linear",     # Öğrenme oranı zamanlayıcısı: doğrusal (lineer bir şekilde azalır, değişmedi)
        seed=3407,                      # Rastgele tohum: 3407 (tekrarlanabilirlik için, değişmedi)
        output_dir=CHECKPOINT_DIR,      # Çıktıların kaydedileceği dizin: "CHECKPOINT_DIR" (değişmedi)
        report_to="none",               # Eğitim raporlama: "none" (WandB gibi araçlar kullanılmayacak, değişmedi)
    ),                                  # Eğitim argümanlarını tamamlar
)                                       # Trainer nesnesini oluşturur

Applying chat template to train dataset (num_proc=2):   0%|          | 0/2324213 [00:03<?, ? examples/s]


KeyError: 'attention_mask'

In [ ]:
tokenizer.save_pretrained(f"{CHECKPOINT_DIR}/final_model2")

In [ ]:
# ----------------------------------------------
# 9. Eğitimi Başlat
# ----------------------------------------------
try:                                    # Hata yakalama bloğu başlatır, checkpoint ile devam etmeyi dener
    print("Checkpoint aranıyor...")     # Kullanıcıya checkpoint’in arandığını bildirir
    trainer.train(resume_from_checkpoint=True)  # Eğitime son checkpoint’ten devam etmeyi dener (önceki bir eğitim varsa)
except Exception as e:                  # Eğer bir hata oluşursa (örneğin, checkpoint bulunamazsa) bu blok çalışır
    print(f"Hata: {str(e)[:200]}...\nYeni eğitim başlatılıyor...")  # Hatanın ilk 200 karakterini yazdırır ve yeni eğitime geçeceğini bildirir
    trainer.train()                     # Sıfırdan yeni bir eğitim başlatır (checkpoint olmadan)



In [ ]:
# ----------------------------------------------
# 10. Modeli Kaydet
# ----------------------------------------------
model.save_pretrained(f"{CHECKPOINT_DIR}/final_model2")  # Eğitilmiş modeli belirtilen dizine kaydeder (CHECKPOINT_DIR/final_model)
tokenizer.save_pretrained(f"{CHECKPOINT_DIR}/final_model2")  # Tokenizer’ı aynı dizine kaydeder

# ----------------------------------------------
# 11. Opsiyonel: Eğitim İlerlemesini İzleme ve Log Kaydetme
# ----------------------------------------------
trainer.save_model(f"{CHECKPOINT_DIR}/final_checkpoin2t")  # Eğitim sırasında ara checkpoint’leri kaydet (opsiyonel, büyük veri seti için faydalı)
print(f"Eğitim tamamlandı. Model şu adreste kaydedildi: {CHECKPOINT_DIR}/final_mode2")